In [1]:
# Load csv data

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import json
import spacy 
from spacy.matcher import Matcher
import pandas as pd
import re
from collections import Counter
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

filepath = os.path.join(os.getcwd(), 'QueryResults_sample.csv')

stack_posts = pd.read_csv(filepath, sep = ",")
    
print("loaded csv data")

loaded csv data


In [4]:
# clean posts and match nouns, adjectives with spacy
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

pattern = [{'POS': 'ADJ', 'OP': '?'},
           {'OP': '?', 'POS': 'NOUN'}
          ]

wordlist = []

for text in stack_posts["AnswerBody"][:50]:                   
    text = re.sub('<pre(.|\n)*?/pre>|<code>(.|\n)*?</code>|(<a(.*|\n)a>)|<blockquote(.|\n)*?\/blockquote>|(<(p|\/p|br|sub|\/sub|em|\/em)>)', '', text)        
    
    doc = nlp(text)
    
    matcher.add("match_nouns", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        wordlist.append(doc[start:end].text)       

c = Counter(wordlist)
print("finished")

finished


In [6]:
# get n relevant words from single document
n = 3 # top n features

stack_answers = stack_posts["AnswerBody"][:50]

text_list = []
for reg_text in stack_answers:    
    reg_text = re.sub('<pre(.|\n)*?/pre>|<code>(.|\n)*?</code>|(<a(.*|\n)a>)|<blockquote(.|\n)*?\/blockquote>|(<(p|\/p|br|sub|\/sub|em|\/em|li|\/li|ul|\/ul|strong|\/strong)>|\n)', '', reg_text)  
    text_list.append(reg_text)


wordlist_vocab = set(wordlist)

top_n = []
for i, text in enumerate(text_list):
    vectorizer = TfidfVectorizer(stop_words = 'english', min_df = 0.01, vocabulary=wordlist_vocab) #, max_features=3
    text_vectorizer = vectorizer.fit_transform([text_list[i]])
    feature_names = vectorizer.get_feature_names()
    
    # sort features and add top n features to list
    feature_array = np.array(vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(text_vectorizer.toarray()).flatten()[::-1]
    top_n.append(feature_array[tfidf_sorting][:n])  

# create vocab set from top_n numpy array    
vocab = set()
for array in top_n:
    for item in array:
        vocab.add(item)

        
print("finished")

finished


In [9]:
# Document similarity
vector_array = np.empty((len(text_list), nlp.vocab.vectors_length))


for i, text in enumerate(text_list):
    doc = nlp(text)
    vector_array[i] = doc.vector


cosine_similarity([vector_array[0]], [vector_array[3]])[0][0]

0.8868453828976748

In [28]:
# Term similarity
# first try
document_term_vectors = []



for i, document in enumerate(text_list):
    doc = nlp(document)
    noun_adj_count = 0
    for token in doc:
        if token.pos_ in ("NOUN", "ADJ"):
            noun_adj_count += 1
    vector_array = np.empty((noun_adj_count, nlp.vocab.vectors_length))    
    index = 0
    for token in doc:        
        if token.pos_ in ("NOUN", "ADJ"):
            vector_array[index] = token.vector
            index += 1
    document_term_vectors.append(vector_array)
#cosine_similarity([vector_array[0]], [vector_array[3]])

200

In [11]:
# Term similarity
# second try

term_term_matrix = []
# loop through relavant words and get their vectors

for i, term in enumerate(vocab):
    term_vector = []    
    doc1 = nlp(str(term))
    vector_base = doc1.vector    
    for j, term in enumerate(vocab):
        doc2 = nlp(str(term))
        vector = doc2.vector
        # add these vectors to term_vector of this word
        term_vector.append(cosine_similarity([vector_base], [vector])[0][0])
    # add vectors to matrix of vectors of all words
    term_term_matrix.append(term_vector)



In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ttm = pd.DataFrame(term_term_matrix, columns = vocab, index = vocab)
ttm

,kernel,string,numbers,metrics,answers,updates,distribution,client,feature,range,mean,case,features,hyperplane,continuous,model,value,alignment,workaround,characters,westons,test,steps,problem,parantheses,bit,instances,objects,sparse,dummy,reducer,new,mind,validation,values,setup,module,thing,svm,items,port,use,line,default,matrix,small,distance,online,frequent,means,complicated,pixel,orthogonal,directory,sklearn,imputation,code,syntax,data,optimization,fit,list,situations,key,labels,image,mapping,counts,proper,training,size,targets,xmean,server,variable,way,algorithm,sure,word,single,quantile,error,projection,example,underestimations,order,iris,classifier,arrangements,grid,file,good,set,classifiers,intro,prior,non,gradient,class,structure,uppercase,learning,loss,number,pipeline,noise,step
kernel,1.000000,0.219056,0.127960,0.204862,0.119206,0.329871,0.274471,0.282890,0.165450,0.059390,0.164372,0.150280,0.144871,0.213102,0.151586,0.131482,0.178345,0.178219,0.506440,0.137043,0.0,0.208171,0.156651,0.363449,0.054235,0.270650,0.264394,0.209810,0.176661,0.282625,0.097837,0.114344,0.176805,0.252752,0.198009,0.367043,0.417641,0.194141,0.139204,0.055279,0.316400,0.229196,0.151866,0.431096,0.289900,0.162650,0.014608,-0.051019,0.027153,0.160272,0.125053,0.236050,0.251843,0.282069,0.0,0.234714,0.353580,0.342302,0.288416,0.282523,0.082096,0.205216,0.121343,0.170452,0.063389,0.211419,0.301759,0.088205,0.197732,0.064975,0.213593,0.121784,0.0,0.411581,0.292266,0.151926,0.384729,0.210526,0.100401,0.154110,0.251843,0.437522,0.140186,0.199312,0.000337,0.100036,0.062104,0.384729,0.001115,0.157647,0.357147,0.132957,0.157684,0.338209,0.082848,0.147650,0.148877,0.099857,0.082508,0.218876,-0.051359,0.093538,0.077024,0.141406,0.152641,0.159747,0.174704
string,0.219056,1.000000,0.381008,0.156219,0.148439,0.124292,0.111320,0.217729,0.252759,0.221656,0.184927,0.292127,0.202440,0.230339,0.250967,0.215282,0.379406,0.247045,0.258707,0.341464,0.0,0.211772,0.191344,0.273571,0.335723,0.318553,0.345902,0.398396,0.226195,0.351730,0.272904,0.236940,0.185917,0.277746,0.332598,0.280964,0.282966,0.262907,0.004497,0.233439,0.225159,0.303962,0.376720,0.384764,0.268694,0.280600,0.195206,0.077969,0.180026,0.213001,0.230796,0.215275,0.250649,0.277297,0.0,0.154364,0.345809,0.468132,0.301880,0.156402,0.232140,0.317033,0.182884,0.388761,0.255963,0.210028,0.234865,0.214794,0.255361,0.089852,0.313932,0.183433,0.0,0.251034,0.464490,0.271850,0.286756,0.248341,0.291767,0.300834,0.250649,0.341313,0.216254,0.367070,-0.067083,0.235840,0.117637,0.286756,0.207433,0.298810,0.364419,0.197062,0.441965,0.222930,0.225154,0.155201,0.200448,0.169762,0.304909,0.326386,0.169818,0.183201,0.137626,0.403231,0.186357,0.221699,0.247251
numbers,0.127960,0.381008,1.000000,0.371299,0.334221,0.264689,0.330841,0.212055,0.266593,0.340347,0.407491,0.288777,0.219008,0.187244,0.209873,0.229281,0.424413,0.210846,0.185390,0.332397,0.0,0.271317,0.261301,0.398845,0.277297,0.332516,0.347396,0.267312,0.175868,0.237811,0.099772,0.245134,0.291718,0.232659,0.430184,0.264733,0.193908,0.332220,-0.074191,0.288631,0.190365,0.339926,0.429218,0.297232,0.283706,0.337496,0.259944,0.271126,0.278253,0.364019,0.297484,0.239449,0.240004,0.241203,0.0,0.292869,0.349709,0.214066,0.480681,0.191717,0.251421,0.430705,0.312880,0.431769,0.300315,0.215313,0.250334,0.477911,0.288574,0.164513,0.360916,0.321726,0.0,0.212008,0.312876,0.357593,0.297131,0.406112,0.363630,0.339247,0.240004,0.334410,0.247397,0.448040,0.180783,0.368393,0.080959,0.297131,0.238481,0.308307,0.301228,0.307129,0.343471,0.271521,0.108433,0.306834,0.215188,0.163524,0.241673,0.273922,0.177364,0.211502,0.248378,0.820055,0.193078,0.246207,0.290087
metrics,0.204862,0.156219,0.371299,1.000000,0.225132,0.307880,0.350007,0.362006,0.188041,0.201633,0.295250,0.145010,0.175025,0.213002,0.326642,0.278037,0.453120,0.358380,0.258988,0.121691,0.0,0.277839,0.258687,0.218011,0.133810,0.137096,0.332530,0.254885,0.114338,0.178227,0.113705,0.145619,0.171323,0.445407,0.427729,0.270325,0.